In [13]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv('input/drug_purchase_500K_d.csv', delimiter="|", header=None)
df.columns = ['memberid', 'memberage', 'membergender', 'pharmacyname', 'pharmcity', 'pharmacystate', 'pharmzip', 'pharmlat', 'pharmlong', 'ndc11code', 'drug_productname', 'drug_ahfsclass', 'drug_shortname', 'drug_indication', 'calcawpunitprice', 'drug_qty', 'daysupply', 'calcawpfullprice', 'netplancost', 'filldate']
display(df.dtypes)
display(df.head())



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,7,8,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


memberid              int64
memberage            object
membergender         object
pharmacyname         object
pharmcity            object
pharmacystate        object
pharmzip             object
pharmlat             object
pharmlong            object
ndc11code             int64
drug_productname     object
drug_ahfsclass       object
drug_shortname       object
drug_indication      object
calcawpunitprice     object
drug_qty             object
daysupply             int64
calcawpfullprice    float64
netplancost         float64
filldate             object
dtype: object

,memberid,memberage,membergender,pharmacyname,pharmcity,pharmacystate,pharmzip,pharmlat,pharmlong,ndc11code,drug_productname,drug_ahfsclass,drug_shortname,drug_indication,calcawpunitprice,drug_qty,daysupply,calcawpfullprice,netplancost,filldate
0,16036852,0,Not Provided,PRIME THERAPEUTICS LLC,IRVING,TX,75063,32.912630,-96.984472,603616132,TRAZODONE TAB 100MG,Serotonin Modulators,TRAZODONE,Depression,0.5769,1,90,103.833,0.00,2015-01-01
1,16035476,0,Not Provided,KROGER PHARMACY,ELIZABETHTOWN,KY,42701,37.734076,-85.887275,61314001510,OFLOXACIN DRO 0.3%OTIC,Antibacterials,OFLOXACIN,Ear Agents,12.7000,1,20,127.000,19.32,2015-01-01
2,16035473,0,Not Provided,WAL-MART PHARMACY 10-0092,AURORA,MO,65605,36.947648,-93.718571,456140530,BYSTOLIC TAB 5MG,beta-Adrenergic Blocking Agents,BYSTOLIC,Hypertension,3.4163,1,30,102.490,52.96,2015-01-01
3,15855700,0,Not Provided,WALGREENS #4123,ELIZABETHTOWN,KY,42701,37.757905,-85.898297,68382041210,OMEPRAZOLE CAP 20MG,Proton-pump Inhibitors,OMEPRAZOLE,Ulcer,4.1515,1,30,124.545,0.00,2015-01-01
4,16035480,0,Not Provided,RITE AID PHARMACY 11663,FORT MILLS,SC,29708,35.060102,-80.991619,93715298,SIMVASTATIN TAB 5MG,HMG-CoA Reductase Inhibitors,SIMVASTATIN,High Cholesterol,2.1043,1,90,189.390,0.00,2015-01-01


In [21]:
# pd.options.display.max_rows = 4000
display(df['drug_indication'].value_counts());
df['drug_indication'].value_counts().shape;

df['membergender'].value_counts()


Hypertension                       46307
Depression                         45399
Bacterial Infection                44001
High Cholesterol                   23629
Pain - Opioid                      22831
Contraception                      22530
Diabetes                           21932
Asthma                             20766
Thyroid Treatment                  18583
Ulcer                              18181
ADHD                               16738
Seizures                           14804
Anxiety                            12598
Inflammation - NSAID               11159
Inflammation                       10788
Cold                               10317
Diuretic                            9287
Hormone Treatment                   8238
Muscle Relaxant                     7500
Insomnia                            6952
Opthalmic Agents                    6452
Cough/Cold/Allergy                  5759
Topical Corticosteroids             4918
Schizophrenia                       4738
Nausea/Vomiting 

Not Provided    503765
Name: membergender, dtype: int64

In [7]:
display(df.shape)

df['drug_name'] = df['drug_shortname'].str.strip()
df.dropna(axis=0, subset=['memberid'], inplace=True)
df['person_code'] = df['memberid'].astype('str')
# df = df[~df['InvoiceNo'].str.contains('C')]
display(df.shape)
df.head()

basket = (df
          .groupby(['person_code', 'drug_name'])['drug_qty']
          .count().unstack().reset_index().fillna(0)
          .set_index('person_code'))

basket.head(10)

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
display(basket_sets.shape)
basket_sets.head()

(503765, 21)

(503765, 21)

(26925, 1956)

drug_name,,1ML SYRINGE,1ML TB,20ML SYRINGE,3ML LL,3ML SYRINGE,5-METHYLTETR,ABACAVIR,ABILIFY,ABSORICA,...,ZOVIRAX,ZUBSOLV,ZUPLENZ,ZYCLARA,ZYLET SUS,ZYMAXID,ZYPREXA,ZYTIGA,ZYVOX TAB,\N
person_code,,,,,,,,,,,,,,,,,,,,,
13881003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13881019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13881049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13881072,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13881106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
frequent_itemsets = apriori(basket_sets, min_support=0.04, use_colnames=True)
# frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# frequent_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 2)]
frequent_itemsets

rules = association_rules(frequent_itemsets, metric="lift")
display(rules.shape)
rules.sort_values(['lift', 'confidence', 'support'], ascending=[False, False, False])

(10, 9)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(PREDNISONE),(AZITHROMYCIN),0.135525,0.191049,0.053222,0.392710,2.055545,0.027330,1.332067
9,(AZITHROMYCIN),(PREDNISONE),0.191049,0.135525,0.053222,0.278577,2.055545,0.027330,1.198292
1,(AMOX/K),(AMOXICILLIN),0.138124,0.232683,0.050028,0.362194,1.556597,0.017889,1.203057
0,(AMOXICILLIN),(AMOX/K),0.232683,0.138124,0.050028,0.215004,1.556597,0.017889,1.097936
4,(AMOXICILLIN),(HYDROCO/APAP),0.232683,0.166314,0.053370,0.229370,1.379137,0.014672,1.081823
5,(HYDROCO/APAP),(AMOXICILLIN),0.166314,0.232683,0.053370,0.320902,1.379137,0.014672,1.129906
6,(HYDROCO/APAP),(AZITHROMYCIN),0.166314,0.191049,0.040371,0.242742,1.270575,0.008597,1.068264
7,(AZITHROMYCIN),(HYDROCO/APAP),0.191049,0.166314,0.040371,0.211314,1.270575,0.008597,1.057057
3,(AZITHROMYCIN),(AMOXICILLIN),0.191049,0.232683,0.050994,0.266913,1.147108,0.006540,1.046692
2,(AMOXICILLIN),(AZITHROMYCIN),0.232683,0.191049,0.050994,0.219154,1.147108,0.006540,1.035993
